In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from tqdm import tqdm
from scipy.special import softmax 
import torch.backends.cudnn as cudnn
import random
import pickle
import pandas as pd
import torch.utils.data as tdata
from scipy.stats import median_abs_deviation as mad
import itertools

In [8]:
import os
import shutil
from google.colab import drive
drive.mount('/content/drive')
checkpoints = '/content/drive/My Drive/Colab Notebooks/cse559a/'
if not os.path.exists(checkpoints):
  os.makedirs(checkpoints)
  
if not os.path.exists('imagenet_val'):
  if not os.path.exists(checkpoints + 'imagenet_val.tar'):
    print("Downloading archive...")
    os.chdir(checkpoints)
    !wget -nv -O imagenet_val.tar -L https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
    os.chdir('/content/')
  print("Copying to local runtime...")
  shutil.copy(checkpoints + 'imagenet_val.tar', './imagenet_val.tar')
  print("Uncompressing...")
  !mkdir imagenet_val
  !tar -xf imagenet_val.tar -C ./imagenet_val/
  !rm imagenet_val.tar
  os.chdir('./imagenet_val') 
  !wget -qO- https://raw.githubusercontent.com/soumith/imagenetloader.torch/master/valprep.sh | bash
  os.chdir('/content/')
print("Data ready!")

if not os.path.exists('exp_v0.py'):
  shutil.copy(checkpoints + 'exp_v0.py', './exp_v0.py')
  print("File imported")
from exp_v0 import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Data ready!


In [9]:
class ConformalModelLogits(nn.Module):
    def __init__(self, model, calib_loader, alpha, kreg=None, lamda=None, randomized=True, allow_zero_sets=False, naive=False, LAC=False, pct_paramtune = 0.3, batch_size=32, lamda_criterion='size'):
        super(ConformalModelLogits, self).__init__()
        self.model = model 
        self.alpha = alpha
        self.randomized = randomized
        self.LAC = LAC
        self.allow_zero_sets = allow_zero_sets
        self.T = platt_logits(self, calib_loader)

        if (kreg == None or lamda == None) and not naive and not LAC:
            kreg, lamda, calib_logits = pick_parameters(model, calib_loader.dataset, alpha, kreg, lamda, randomized, allow_zero_sets, pct_paramtune, batch_size, lamda_criterion)
            calib_loader = tdata.DataLoader(calib_logits, batch_size=batch_size, shuffle=False, pin_memory=True)

        self.penalties = np.zeros((1, calib_loader.dataset[0][0].shape[0]))
        if not (kreg == None) and not naive and not LAC:
            self.penalties[:, kreg:] += lamda
        self.Qhat = 1-alpha
        if not naive and not LAC:
            self.Qhat = conformal_calibration_logits(self, calib_loader)
        elif not naive and LAC:
            gt_locs_cal = np.array([np.where(np.argsort(x[0]).flip(dims=(0,)) == x[1])[0][0] for x in calib_loader.dataset])
            scores_cal = 1-np.array([np.sort(torch.softmax(calib_loader.dataset[i][0]/self.T.item(), dim=0))[::-1][gt_locs_cal[i]] for i in range(len(calib_loader.dataset))]) 
            self.Qhat = np.quantile( scores_cal , np.ceil((scores_cal.shape[0]+1) * (1-alpha)) / scores_cal.shape[0] )

    def forward(self, logits, randomized=None, allow_zero_sets=None):
        if randomized == None:
            randomized = self.randomized
        if allow_zero_sets == None:
            allow_zero_sets = self.allow_zero_sets
        
        with torch.no_grad():
            logits_numpy = logits.detach().cpu().numpy()
            scores = softmax(logits_numpy/self.T.item(), axis=1)

            if not self.LAC:
                I, ordered, cumsum = sort_sum(scores)

                S = gcq(scores, self.Qhat, I=I, ordered=ordered, cumsum=cumsum, penalties=self.penalties, randomized=randomized, allow_zero_sets=allow_zero_sets)
            else:
                S = [ np.where( (1-scores[i,:]) < self.Qhat )[0] for i in range(scores.shape[0]) ]

        return logits, S

def conformal_calibration_logits(cmodel, calib_loader):
    with torch.no_grad():
        E = np.array([])
        for logits, targets in calib_loader:
            logits = logits.detach().cpu().numpy()

            scores = softmax(logits/cmodel.T.item(), axis=1)

            I, ordered, cumsum = sort_sum(scores)

            E = np.concatenate((E,giq(scores,targets,I=I,ordered=ordered,cumsum=cumsum,penalties=cmodel.penalties,randomized=True,allow_zero_sets=True)))
            
        # Qhat = np.quantile(E,1-cmodel.alpha,interpolation='higher')
        Qhat = np.quantile(E, 1 - cmodel.alpha, method='higher')

        return Qhat 

def platt_logits(cmodel, calib_loader, max_iters=10, lr=0.01, epsilon=0.01):
    nll_criterion = nn.CrossEntropyLoss().cuda()

    T = nn.Parameter(torch.Tensor([1.3]).cuda())

    optimizer = optim.SGD([T], lr=lr)
    for iter in range(max_iters):
        T_old = T.item()
        for x, targets in calib_loader:
            optimizer.zero_grad()
            x = x.cuda()
            x.requires_grad = True
            out = x/T
            loss = nll_criterion(out, targets.long().cuda())
            loss.backward()
            optimizer.step()
        if abs(T_old - T.item()) < epsilon:
            break
    return T 

### CORE CONFORMAL INFERENCE FUNCTIONS

# Generalized conditional quantile function.
def gcq(scores, tau, I, ordered, cumsum, penalties, randomized, allow_zero_sets):
    penalties_cumsum = np.cumsum(penalties, axis=1)
    sizes_base = ((cumsum + penalties_cumsum) <= tau).sum(axis=1) + 1  # 1 - 1001
    sizes_base = np.minimum(sizes_base, scores.shape[1]) # 1-1000

    if randomized:
        V = np.zeros(sizes_base.shape)
        for i in range(sizes_base.shape[0]):
            V[i] = 1/ordered[i,sizes_base[i]-1] * \
                    (tau-(cumsum[i,sizes_base[i]-1]-ordered[i,sizes_base[i]-1])-penalties_cumsum[0,sizes_base[i]-1]) # -1 since sizes_base \in {1,...,1000}.

        sizes = sizes_base - (np.random.random(V.shape) >= V).astype(int)
    else:
        sizes = sizes_base

    if tau == 1.0:
        sizes[:] = cumsum.shape[1] # always predict max size if alpha==0. (Avoids numerical error.)

    if not allow_zero_sets:
        sizes[sizes == 0] = 1 # allow the user the option to never have empty sets (will lead to incorrect coverage if 1-alpha < model's top-1 accuracy

    S = list()

    # Construct S from equation (5)
    for i in range(I.shape[0]):
        S = S + [I[i,0:sizes[i]],]

    return S

# Get the 'p-value'
def get_tau(score, target, I, ordered, cumsum, penalty, randomized, allow_zero_sets): # For one example
    idx = np.where(I==target)
    tau_nonrandom = cumsum[idx]

    if not randomized:
        return tau_nonrandom + penalty[0]
    
    U = np.random.random()

    if idx == (0,0):
        if not allow_zero_sets:
            return tau_nonrandom + penalty[0]
        else:
            return U * tau_nonrandom + penalty[0] 
    else:
        return U * ordered[idx] + cumsum[(idx[0],idx[1]-1)] + (penalty[0:(idx[1][0]+1)]).sum()

# Gets the histogram of Taus. 
def giq(scores, targets, I, ordered, cumsum, penalties, randomized, allow_zero_sets):
    """
        Generalized inverse quantile conformity score function.
        E from equation (7) in Romano, Sesia, Candes.  Find the minimum tau in [0, 1] such that the correct label enters.
    """
    E = -np.ones((scores.shape[0],))
    for i in range(scores.shape[0]):
        E[i] = get_tau(scores[i:i+1,:],targets[i].item(),I[i:i+1,:],ordered[i:i+1,:],cumsum[i:i+1,:],penalties[0,:],randomized=randomized, allow_zero_sets=allow_zero_sets)

    return E

### AUTOMATIC PARAMETER TUNING FUNCTIONS
def pick_kreg(paramtune_logits, alpha):
    gt_locs_kstar = np.array([np.where(np.argsort(x[0]).flip(dims=(0,)) == x[1])[0][0] for x in paramtune_logits])
    #kstar = np.quantile(gt_locs_kstar, 1-alpha, interpolation='higher') + 1
    kstar = np.quantile(gt_locs_kstar, 1-alpha, method='higher') + 1
    return kstar 

def pick_lamda_size(model, paramtune_loader, alpha, kreg, randomized, allow_zero_sets):
    # Calculate lamda_star
    best_size = iter(paramtune_loader).__next__()[0][1].shape[0] # number of classes 
    # Use the paramtune data to pick lamda.  Does not violate exchangeability.
    for temp_lam in [0.001, 0.01, 0.1, 0.2, 0.5]: # predefined grid, change if more precision desired.
        conformal_model = ConformalModelLogits(model, paramtune_loader, alpha=alpha, kreg=kreg, lamda=temp_lam, randomized=randomized, allow_zero_sets=allow_zero_sets, naive=False)
        top1_avg, top5_avg, cvg_avg, sz_avg = validate(paramtune_loader, conformal_model, print_bool=False)
        if sz_avg < best_size:
            best_size = sz_avg
            lamda_star = temp_lam
    return lamda_star

def pick_lamda_adaptiveness(model, paramtune_loader, alpha, kreg, randomized, allow_zero_sets, strata=[[0,1],[2,3],[4,6],[7,10],[11,100],[101,1000]]):
    # Calculate lamda_star
    lamda_star = 0
    best_violation = 1
    # Use the paramtune data to pick lamda.  Does not violate exchangeability.
    for temp_lam in [0, 1e-5, 1e-4, 8e-4, 9e-4, 1e-3, 1.5e-3, 2e-3]: # predefined grid, change if more precision desired.
        conformal_model = ConformalModelLogits(model, paramtune_loader, alpha=alpha, kreg=kreg, lamda=temp_lam, randomized=randomized, allow_zero_sets=allow_zero_sets, naive=False)
        curr_violation = get_violation(conformal_model, paramtune_loader, strata, alpha)
        if curr_violation < best_violation:
            best_violation = curr_violation 
            lamda_star = temp_lam
    return lamda_star

def pick_parameters(model, calib_logits, alpha, kreg, lamda, randomized, allow_zero_sets, pct_paramtune, batch_size, lamda_criterion):
    num_paramtune = int(np.ceil(pct_paramtune * len(calib_logits)))
    paramtune_logits, calib_logits = tdata.random_split(calib_logits, [num_paramtune, len(calib_logits)-num_paramtune])
    calib_loader = tdata.DataLoader(calib_logits, batch_size=batch_size, shuffle=False, pin_memory=True)
    paramtune_loader = tdata.DataLoader(paramtune_logits, batch_size=batch_size, shuffle=False, pin_memory=True)

    if kreg == None:
        kreg = pick_kreg(paramtune_logits, alpha)
    if lamda == None:
        if lamda_criterion == "size":
            lamda = pick_lamda_size(model, paramtune_loader, alpha, kreg, randomized, allow_zero_sets)
        elif lamda_criterion == "adaptiveness":
            lamda = pick_lamda_adaptiveness(model, paramtune_loader, alpha, kreg, randomized, allow_zero_sets)
    return kreg, lamda, calib_logits

def get_violation(cmodel, loader_paramtune, strata, alpha):
    df = pd.DataFrame(columns=['size', 'correct'])
    for logit, target in loader_paramtune:
        # compute output
        output, S = cmodel(logit) # This is a 'dummy model' which takes logits, for efficiency.
        # measure accuracy and record loss
        size = np.array([x.size for x in S])
        I, _, _ = sort_sum(logit.numpy()) 
        correct = np.zeros_like(size)
        for j in range(correct.shape[0]):
            correct[j] = int( target[j] in list(S[j]) )
        batch_df = pd.DataFrame({'size': size, 'correct': correct})
        df = pd.concat([df,batch_df], ignore_index=True)
    wc_violation = 0
    for stratum in strata:
        temp_df = df[ (df['size'] >= stratum[0]) & (df['size'] <= stratum[1]) ]
        if len(temp_df) == 0:
            continue
        stratum_violation = abs(temp_df.correct.mean()-(1-alpha))
        wc_violation = max(wc_violation, stratum_violation)
    return wc_violation # the violation

In [10]:
def make_table(df):
  round_to_n = lambda x, n: np.round(x, -int(np.floor(np.log10(x))) + (n - 1)) # Rounds to sig figs
  table = ""
  table += "\\begin{table}[t]\n"
  table += "\\centering\n"
  table += "\\small\n"
  table += "\\begin{tabular}{lcccc}\n"
  table += "\\toprule \n"
  table += " & \\multicolumn{2}{c}{Violation at $\\alpha=10\\%$}  & \\multicolumn{2}{c}{Violation at $\\alpha=5\\%$} \\\\ \n"
  table += "\\cmidrule(r){2-3}  \\cmidrule(r){4-5} \n"
  table += "Model & APS & RAPS & APS & RAPS \\\\ \n"
  table += "\\midrule \n"

  for model in df.Model.unique():
      dfmodel = df[df.Model == model]
      table += model + " & "
      table += str(np.round(dfmodel[dfmodel.alpha == 0.1]["APS violation"].item(),3)) + " & "
      table += "\\bf " + str(np.round(dfmodel[dfmodel.alpha == 0.1]["RAPS violation"].item(),3)) + " & "
      table += str(np.round(dfmodel[dfmodel.alpha == 0.05]["APS violation"].item(),3)) + " & "
      table += "\\bf " + str(np.round(dfmodel[dfmodel.alpha == 0.05]["RAPS violation"].item(),3)) + " " 
      table += "\\\\ \n"
  
  table += "\\bottomrule \n"
  table += "\\end{tabular} \n"
  table += "\\caption{\\textbf{Adaptiveness results after automatically tuning $\\lambda$.} We report the median worst-stratum coverage violations of \\aps\\ and \\raps\\ over 10 trials. See Appendix~\\ref{app:optimizing-adaptiveness} for experimental details.} \n"
  table += "\\label{table:tunelambda} \n"
  table += "\\end{table} \n"
  return table    

# Returns a dataframe with:
# 1) Set sizes for all test-time examples.
# 2) topk for each example, where topk means which score was correct.
def get_worst_violation(modelname, datasetname, datasetpath, alpha, strata, randomized, n_data_conf, n_data_val, pct_paramtune, bsz):
  ### Data Loading
  logits = get_logits_dataset(modelname, datasetname, datasetpath)
  calib_logits, val_logits = tdata.random_split(logits, [n_data_conf, len(logits)-n_data_conf]) # A new random split for every trial
  # Prepare the loaders
  calib_loader = torch.utils.data.DataLoader(calib_logits, batch_size = bsz, shuffle=False, pin_memory=True)
  val_loader = torch.utils.data.DataLoader(val_logits, batch_size = bsz, shuffle=False, pin_memory=True)

  ### Instantiate and wrap model
  model = create_model(modelname)
  # Conformalize the model with the APS parameter choice
  conformal_model = ConformalModelLogits(model, calib_loader, alpha=alpha, kreg=0, lamda=0, randomized=randomized, allow_zero_sets=True, naive=False)
  aps_worst_violation = get_violation(conformal_model, val_loader, strata, alpha)
  # Conformalize the model with an optimal parameter choice
  conformal_model = ConformalModelLogits(model, calib_loader, alpha=alpha, kreg=None, lamda=None, randomized=randomized, allow_zero_sets=True, naive=False, pct_paramtune=pct_paramtune, lamda_criterion='adaptiveness')
  raps_worst_violation = get_violation(conformal_model, val_loader, strata, alpha)

  return aps_worst_violation, raps_worst_violation 

def experiment(modelname, datasetname, datasetpath, num_trials, alpha, strata, randomized, n_data_conf, n_data_val, pct_paramtune, bsz):
  _fix_randomness()
  ### Data Loading
  logits = get_logits_dataset(modelname, datasetname, datasetpath)

  ### Instantiate and wrap model
  model = create_model(modelname)

  ### Perform experiment
  aps_violations = np.zeros((num_trials,))
  raps_violations = np.zeros((num_trials,))
  for i in tqdm(range(num_trials)):
      aps_violations[i], raps_violations[i] = get_worst_violation(modelname, datasetname, datasetpath, alpha, strata, randomized, n_data_conf, n_data_val, pct_paramtune, bsz)
      print(f'\n\tAPS Violation: {np.median(aps_violations[0:i+1]):.3f}, RAPS Violation: {np.median(raps_violations[0:i+1]):.3f}\033[F', end='')
  print('')
  return np.median(aps_violations), np.median(raps_violations)

def _fix_randomness(seed=0):
  ### Fix randomness 
  np.random.seed(seed=seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  random.seed(seed)


  

In [11]:
def trial(model, logits, alpha, kreg, lamda, randomized, n_data_conf, n_data_val, bsz, naive_bool):
    logits_cal, logits_val = split2(logits, n_data_conf, n_data_val) # A new random split for every trial
    # Prepare the loaders
    loader_cal = torch.utils.data.DataLoader(logits_cal, batch_size = bsz, shuffle=False, pin_memory=True)
    loader_val = torch.utils.data.DataLoader(logits_val, batch_size = bsz, shuffle=False, pin_memory=True)
    # Conformalize the model
    conformal_model = ConformalModelLogits(model, loader_cal, alpha=alpha, kreg=kreg, lamda=lamda, randomized=randomized, allow_zero_sets=True, naive=naive_bool)
    # Collect results
    top1_avg, top5_avg, cvg_avg, sz_avg = validate(loader_val, conformal_model, print_bool=False)
    return top1_avg, top5_avg, cvg_avg, sz_avg

def experiment(modelname, datasetname, datasetpath, num_trials, alpha, kreg, lamda, randomized, n_data_conf, n_data_val, bsz, predictor):
    ### Experiment logic
    naive_bool = predictor == 'Naive'
    if predictor in ['Naive', 'APS']:
        lamda = 0 # No regularization.

    ### Data Loading
    logits = get_logits_dataset(modelname, datasetname, datasetpath)

    ### Instantiate and wrap model
    model = create_model(modelname)

    ### Perform experiment
    top1s = np.zeros((num_trials,))
    top5s = np.zeros((num_trials,))
    coverages = np.zeros((num_trials,))
    sizes = np.zeros((num_trials,))
    for i in tqdm(range(num_trials)):
        top1_avg, top5_avg, cvg_avg, sz_avg = trial(model, logits, alpha, kreg, lamda, randomized, n_data_conf, n_data_val, bsz, naive_bool)
        top1s[i] = top1_avg
        top5s[i] = top5_avg
        coverages[i] = cvg_avg
        sizes[i] = sz_avg
        print(f'\n\tTop1: {np.median(top1s[0:i+1]):.3f}, Top5: {np.median(top5s[0:i+1]):.3f}, Coverage: {np.median(coverages[0:i+1]):.3f}, Size: {np.median(sizes[0:i+1]):.3f}\033[F', end='')
    print('')
    return np.median(top1s), np.median(top5s), np.median(coverages), np.median(sizes), mad(top1s), mad(top5s), mad(coverages), mad(sizes)

def _fix_randomness(seed):
    np.random.seed(seed=seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)

def _format_appendix_table(df):
    kregs = df.kreg.unique()
    lamdas = df["lambda"].unique()
    num_kreg = len(kregs)
    num_lamda = len(lamdas)
    latex_table = '''\\begin{table}[t] \n 
\\centering \n
\\tiny \n
\\begin{tabular}{l'''
    for i in range(num_kreg):
        latex_table += "c"

    latex_table += '''} \n
\\toprule\n
$k_{reg} | \lambda$ & '''
    for i in range(num_lamda):
        latex_table += str(lamdas[i]) + ' '
        if i < (num_lamda - 1):
            latex_table += ' & '

    latex_table += "\\\\\n"
    latex_table += "\\midrule\n"

    for kreg in kregs:
        latex_table += str(kreg) + ' & '
        for i in range(num_lamda):
            latex_table += str(df[(df["lambda"] == lamdas[i]) & (df["kreg"] == kreg)]["Size"].item()) + ' '
            if i < (num_lamda - 1):
                latex_table += ' & '
        latex_table += "\\\\\n"

    latex_table += "\\bottomrule\n"
    latex_table += "\\end{tabular}\n"
    latex_table += "\\caption{Table caption.}\n"
    latex_table += "\\label{table:parameters}\n"
    latex_table += "\\end{table}\n"
        
    return latex_table

In [12]:
### Fix randomness 
_fix_randomness(seed=0)

### Configure experiment
modelnames = ['ResNet152']
alphas = [0.10]
kregs = [1, 2, 5, 10, 50]
lamdas = [0, 0.0001, 0.001, 0.01, 0.02, 0.05, 0.2, 0.5, 0.7, 1.0]
predictors = ['RAPS']
params = list(itertools.product(modelnames, alphas, predictors,kregs, lamdas))
m = len(params)
datasetname = 'Imagenet'
datasetpath = './imagenet_val/'
num_trials = 10
randomized = True
n_data_conf = 2000
n_data_val = 2000
bsz = 64
cudnn.benchmark = True

### Perform the experiment
df = pd.DataFrame(columns = ["Model","Predictor","Top1","Top5","alpha","kreg","lambda","Coverage","Size"])
for i in range(m):
    _fix_randomness(seed=0)
    modelname, alpha, predictor, kreg, lamda = params[i]
    print(f'Model: {modelname} | Desired coverage: {1-alpha} | kreg: {kreg} | lambda: {lamda} ')

    out = experiment(modelname, datasetname, datasetpath, num_trials, params[i][1], kreg, lamda, randomized, n_data_conf, n_data_val, bsz, predictor) 
    new_row = pd.DataFrame([{"Model": modelname,
                    "Predictor": predictor,
                    "Top1": np.round(out[0],3),
                    "Top5": np.round(out[1],3),
                    "alpha": alpha,
                    "kreg": kreg,
                    "lambda": lamda,
                    "Coverage": np.round(out[2],3),
                    "Size": np.round(out[3],3)}])
    df = pd.concat([df, new_row], ignore_index=True)

## Print the TeX table
table = open("./appendix_parameter_table.tex", 'w')
table.write(_format_appendix_table(df))
table.close()

Model: ResNet152 | Desired coverage: 0.9 | kreg: 1 | lambda: 0 


 10%|█         | 1/10 [00:00<00:07,  1.13it/s]


	Top1: 0.781, Top5: 0.945, Coverage: 0.910, Size: 11.450

 20%|██        | 2/10 [00:01<00:06,  1.20it/s]


	Top1: 0.786, Top5: 0.942, Coverage: 0.907, Size: 11.628

...


	Top1: 0.790, Top5: 0.943, Coverage: 0.900, Size: 6.867

 90%|█████████ | 9/10 [00:07<00:00,  1.18it/s]


	Top1: 0.790, Top5: 0.944, Coverage: 0.902, Size: 6.827

100%|██████████| 10/10 [00:08<00:00,  1.22it/s]


	Top1: 0.790, Top5: 0.944, Coverage: 0.900, Size: 6.867
